In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#### Postprocessing Code starts here ####
## الكود مضبوط مرة بس فيه مشكلة في اذا صارت الكلمة مكررة مرتين في النص وموجودة في السبان ياخذ اقرب وحدة ##
## تم التاكد من دقة المخرجات ##
## اكتشفت انه في الكونت يطرح ١ من النهاية ،، شلتها
##        end = start + len(' '.join(tokens)) - 1
import pandas as pd
import re

def find_span_offsets(text, span):
    tokens = re.findall(r'\b[\w,-;>]+\b', span)
    span_offsets = []
    text = text.lower()
    if span == "NA":
      return "NA"
    if ' '.join(tokens) in text:
        # continuous span
        start = text.index(' '.join(tokens))
        end = start + len(' '.join(tokens))
        span_offsets.append(f"{start}-{end}")
    else:
        # disjoint tokens
        i = 0
        while i < len(tokens):
            if tokens[i] in text:
                start = text.find(tokens[i])
                end = start + len(tokens[i])
                # check if the next token is consecutive with the current one
                #old condition: (end + 1 == text.find(tokens[i + 1]) or end + 2 == text.find(tokens[i + 1]))
                while i + 1 < len(tokens) and tokens[i]+ " " +tokens[i+1] in text:
                    i += 1
                    end = end + len(tokens[i]) + 1
                span_offsets.append(f"{start}-{end}")
            i += 1

    return ','.join(span_offsets)


def process_spans(row):
    spans = re.split(r'(?=(?:normf:|keyf:))', row['Output'])
    spans = [span.strip() for span in spans if span.strip()]

    labels = []
    polarities = []
    span_content = []
    span_offsets = []

    for span in spans:
        if 'normf:' in span:
            labels.append('normf:')
            polarities.append('X')
        elif 'keyf:' in span:
            labels.append('keyf:')
            polarities.append('NA')
        else:
            labels.append('NA')
            polarities.append('NA')

        # Remove ';' from the end of the span if it exists
        span = span.replace('normf:', '').replace('keyf:', '').strip()
        if span.endswith(';'):
            span = span[:-1]

        # Check if the span is 'na' (case-insensitive) and replace it with 'NA'
        if span.lower() == 'na':
            span = 'NA'

        # Remove unwanted spaces around dash "-", colon ":", and comma ","
        span = span.replace(' - ', '-').replace(' :', ':').replace(' ,', ',')

        span_content.append(span)
        span_offsets.append(find_span_offsets(row['Text'], span))

    return pd.DataFrame({'ObservationID': [row['ObservationID']] * len(span_content),
                         'Text': [row['Text']] * len(span_content),
                         'Polarity': polarities,
                         'Spans': span_content,
                         'Spans_Off': span_offsets})

# put the file here to preprocess it
input_file = '/content/drive/MyDrive/biocreative/FinalResults/FlanT5xl-LoRa-1024/Merged/TestResults.tsv'
output_file = '1.tsv'
df = pd.read_csv(input_file, delimiter='\t')

new_rows = df.apply(process_spans, axis=1)

# concatenate the new rows and save 
result_df = pd.concat(new_rows.tolist(), ignore_index=True)
result_df.to_csv(output_file, sep='\t', index=False)

print("Processing complete. Results saved in '1.tsv'")


Processing complete. Results saved in '1.tsv'


In [ ]:
# remove the duplicate rows

file_path = '1.tsv'
with open(file_path, 'r') as f:
    rows = [line.strip().split('\t') for line in f]

seen_rows = set()
duplicate_rows = []
unique_rows = []

for row in rows:
    row_tuple = tuple(row)
    if row_tuple in seen_rows:
        duplicate_rows.append(row)
    else:
        seen_rows.add(row_tuple)
        unique_rows.append(row)

output_path = 'TBL_Model.tsv'
with open(output_path, 'w') as f:
    for row in unique_rows:
        f.write('\t'.join(row) + '\n')


In [ ]:
print(duplicate_rows)

[['1daf211987e73562c4399f93da11707c', 'NAILS HAIR SKIN: Left hand: one over knuckle of second finger (red patch 2.5 cm in diameter); smaller patches on lateral side of palm just below 2nd finger; another on thumb on thumb on palmar side; on wrist. Right hand: palmar side, one of 3rd and 4th finger; on other sider, one on 5th finger and one below thumb. Left upper arm: smaller and bigger ones. On right upper arm: larger brown patch. On left upper in though: 3 patches; one on upper right leg; one on right posterior thigh. Right thigh: hyperpigmented area.', 'NA', 'red patches', '63-66,102-109'], ['1daf211987e73562c4399f93da11707c', 'NAILS HAIR SKIN: Left hand: one over knuckle of second finger (red patch 2.5 cm in diameter); smaller patches on lateral side of palm just below 2nd finger; another on thumb on thumb on palmar side; on wrist. Right hand: palmar side, one of 3rd and 4th finger; on other sider, one on 5th finger and one below thumb. Left upper arm: smaller and bigger ones. On r

In [ ]:
print(len(duplicate_rows))

17
